In [ ]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.4 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
! pip install backtesting 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 10.1 MB/s 
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173832 sha256=3830bbadf0e7b5165349a5a9e1cfed7cfd1cca374a45d5a95601f81d6640f1df
  Stored in directory: /root/.cache/pip/wheels/48/1d/ea/65dacebc37da7655d8a1fd0f315ac39d102e31d6545237a1c6
Successfully built backtesting


In [ ]:
import pandas as pd

def SMA(array, n):
    """Simple moving average"""
    return pd.Series(array).rolling(n).mean()

def RSI(array, n):
    """Relative strength index"""
    # Approximate; good enough
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)

In [ ]:
from backtesting import Strategy, Backtest
from backtesting.lib import resample_apply


class System(Strategy):
    d_rsi = 30  # Daily RSI lookback periods
    w_rsi = 30  # Weekly
    level = 70
    
    def init(self):
        # Compute moving averages the strategy demands
        self.ma10 = self.I(SMA, self.data.Close, 10)
        self.ma20 = self.I(SMA, self.data.Close, 20)
        self.ma50 = self.I(SMA, self.data.Close, 50)
        self.ma100 = self.I(SMA, self.data.Close, 100)
        
        # Compute daily RSI(30)
        self.daily_rsi = self.I(RSI, self.data.Close, self.d_rsi)
        
        # To construct weekly RSI, we can use `resample_apply()`
        # helper function from the library
        self.weekly_rsi = resample_apply(
            'W-FRI', RSI, self.data.Close, self.w_rsi)
        
        
    def next(self):
        price = self.data.Close[-1]
        
        # If we don't already have a position, and
        # if all conditions are satisfied, enter long.
        if (not self.position and
            self.daily_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.daily_rsi[-1] and
            self.ma10[-1] > self.ma20[-1] > self.ma50[-1] > self.ma100[-1] and
            price > self.ma10[-1]):
            
            # Buy at market price on next open, but do
            # set 8% fixed stop loss.
            self.buy(sl=.92 * price)
        
        # If the price closes 2% or more below 10-day MA
        # close the position, if any.
        elif price < .98 * self.ma10[-1]:
            self.position.close()

/usr/local/lib/python3.7/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [ ]:
import yfinance as yf
GOOGLE=yf.download("GOOG",start="2018-01-01", end="2020-06-30",auto_adjust = True)

from backtesting import Backtest

backtest = Backtest(GOOGLE, System, commission=.002)
stats= backtest.run()

[*********************100%***********************]  1 of 1 completed


In [ ]:
%%time

backtest.optimize(d_rsi=range(10, 35, 5),
                  w_rsi=range(10, 35, 5),
                  level=range(30, 80, 10))

Backtest.optimize:   0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 189 ms, sys: 36.9 ms, total: 226 ms
Wall time: 7.62 s


Start                     2018-01-02 00:00:00
End                       2020-06-29 00:00:00
Duration                    909 days 00:00:00
Exposure Time [%]                    2.711324
Equity Final [$]                 10196.024834
Equity Peak [$]                  10367.342079
Return [%]                           1.960248
Buy & Hold Return [%]               30.983093
Return (Ann.) [%]                     0.78328
Volatility (Ann.) [%]                2.893186
Sharpe Ratio                         0.270733
Sortino Ratio                        0.403246
Calmar Ratio                         0.221502
Max. Drawdown [%]                    -3.53622
Avg. Drawdown [%]                   -1.465625
Max. Drawdown Duration      259 days 00:00:00
Avg. Drawdown Duration      140 days 00:00:00
# Trades                                    2
Win Rate [%]                             50.0
Best Trade [%]                       2.208092
Worst Trade [%]                     -0.235516
Avg. Trade [%]                    

In [ ]:
backtest.plot()


Row(id='1513', ...)

In [ ]:
stats['_trades'].head(5)

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
